# Question answering over a group chat messages
In this tutorial, we are going to use Langchain + Deep Lake with GPT4 to semantically search and ask questions over a group chat.

View a working demo [here](https://twitter.com/thisissukh_/status/1647223328363679745)

## 1. Install required packages

In [ ]:
!python3 -m pip install --upgrade langchain deeplake openai tiktoken

## 2. Add API keys

In [4]:
import os
import getpass
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import DeepLake
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from dotenv import load_dotenv

# os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
# os.environ['ACTIVELOOP_TOKEN'] = getpass.getpass('Activeloop Token:')
# os.environ['ACTIVELOOP_ORG'] = getpass.getpass('Activeloop Org:')
load_dotenv()
org = os.getenv('ACTIVELOOP_ORG')
embeddings = OpenAIEmbeddings()

dataset_path = 'hub://' + org + '/data'



## 2. Create sample data

你可以使用ChatGPT生成一个示例群聊对话，使用以下提示：
```
生成一个群聊对话，三个朋友谈论他们的一天，引用真实的地方和虚构的名字。让它尽可能有趣和详细。
```
我已经在`messages-CN-zh.txt`中生成了这样的聊天。我们可以保持简单，用这个例子来说明。

## 3. Ingest chat embeddings

We load the messages in the text file, chunk and upload to ActiveLoop Vector store.

In [5]:
with open("messages-CN-zh.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
pages = text_splitter.split_text(state_of_the_union)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.create_documents(pages)

print (texts)

dataset_path = 'hub://'+org+'/data'
embeddings = OpenAIEmbeddings()
db = DeepLake.from_documents(texts, embeddings, dataset_path=dataset_path, overwrite=True)

[Document(page_content='当然，以下是 Joey、Rachel 和 Monica 的一段聊天！\n\nRachel：嗨，大家，你们今天过得怎么样？\n\nJoey：还不错，Rach。我今天去了自然历史博物馆。\n\nMonica：哦，我喜欢那个地方！你看到了恐龙展览吗？\n\nJoey：是的，太神奇了！他们有一具巨大的霸王龙骨架，我说：“哇，那个家伙有些严重的咬合力！”\n\nRachel：（笑）Joey 的经典语录。\n\nMonica：那么，Rachel，你今天干了什么？\n\nRachel：嗯，我一整天都在 Central Perk 上班。但是不太糟糕，因为 Gunther 让我在拿铁里加了一份浓缩咖啡。\n\nJoey：（嘿嘿一笑）这就是我的女孩，总是突破极限。\n\nMonica：说到极限，我今天早上去了 Barry 的 Bootcamp，真是疯狂。我感觉我锻炼了身体的每一个肌肉。\n\nRachel：啊，我讨厌锻炼。我们能不能谈些更有趣的事情？\n\nJoey：我知道什么有趣！我今天见了我的老朋友 Chandler。\n\nMonica：Chandler？我好久没见他了！他怎么样？\n\nJoey：他很好。他仍然住在 Tulsa，还在那个数据处理公司工作。\n\nRachel：（讽刺地）哇，听起来很惊险。\n\nJoey：嘿，不要轻视它。Chandler 在那里过得很好。他有房子、有车，还有一只名叫 Yasmine 的宠物鸭子。\n\nMonica：（笑）宠物鸭子？只有 Chandler 才会这么做。\n\nRachel：你知道更疯狂的是什么吗？我听说 Ross 上周末去参加了毛茸茸动物大会。\n\nJoey：（喷出饮料）什么？那太疯狂了！他为什么要这么做？\n\nMonica：我不知道，但我听说他穿着像只巨型松鼠。\n\nRachel：（笑）我甚至无法想象那会是什么样子。\n\nJoey：（摇头）Ross，伙计。他一直有点……奇怪。\n\nMonica：（微笑）这就是我们爱他的原因。\n\nRachel：（微笑着回应）是啊，他是我们奇怪的小朋友。\n\nJoey：（举起酒杯）为奇怪的朋友和快乐的日', metadata={})]
Your Deep Lake dataset has been successfully create

|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/gnehcgnaw/data


hub://gnehcgnaw/data loaded successfully.


Evaluating ingest: 100%|██████████| 1/1 [01:37<00:00
|

Dataset(path='hub://gnehcgnaw/data', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (1, 1536)  float32   None   
    ids      text     (1, 1)      str     None   
 metadata    json     (1, 1)      str     None   
   text      text     (1, 1)      str     None   


## 4. Ask questions

Now we can ask a question and get an answer back with a semantic search:

In [7]:
db = DeepLake(dataset_path=dataset_path, read_only=True, embedding_function=embeddings)

# as_retriever 是一个方法，它将DeepLake转换为Retriever对象，该对象可以用于检索
retriever = db.as_retriever()
# search_kwargs 是一个字典，它将传递给Retriever的search方法 ，distance_metric是用于检索的距离度量， cos是余弦距离
retriever.search_kwargs['distance_metric'] = 'cos'
# k是检索的数量
retriever.search_kwargs['k'] = 4

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=False)

while True:
    query = input("Enter query:")
    if query == "quit":
        break
    # The Hungry Lobster
    ans = qa({"query": query})
    print(ans)

/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/gnehcgnaw/data



|

hub://gnehcgnaw/data loaded successfully.



Deep Lake Dataset in hub://gnehcgnaw/data already exists, loading from the storage
Dataset(path='hub://gnehcgnaw/data', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (1, 1536)  float32   None   
    ids      text     (1, 1)      str     None   
 metadata    json     (1, 1)      str     None   
   text      text     (1, 1)      str     None   
{'query': '总共几个人', 'result': ' 五个人：Joey、Rachel、Monica、Chandler 和 Ross。'}
{'query': '他们讨论了什么？', 'result': ' 他们讨论了 Joey 去参观博物馆、Monica 去参加 Barry 的 Bootcamp 以及 Ross 参加毛茸茸动物大会的故事。'}
{'query': 'Joey说了几句话', 'result': ' Joey说了 5 句话。'}
